<a href="https://colab.research.google.com/github/preyankah/APIs/blob/master/GEE_PythonAPI_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install library
!pip install earthengine-api

In [0]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

ee.Initialize() initializes the API. Required before API is called else a 404 error is thrown

In [0]:
ee.Initialize()

In [12]:
# https://developers.google.com/earth-engine/image_visualization
# Import the Folium library 
# Basemap not automatically added when GEE tiles are added
import folium

# Define a method for displaying Earth Engine image tiles to folium map
# Establish a connection to Google Maps API for adding tile layers from GEE
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Location to extract imagery for using geographic coordinates
point = ee.Geometry.Point([-71.8023,42.2626])
# Dates
startDate = '2018-01-01'
endDate = '2019-12-31'

# Specify time period of interest
fcc = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(point).filterDate(startDate, endDate).min()

# Set visualization parameters
# Check dataset's page for required parameters for visualization
# Define dataset from image collection to visualize in the band list

# ----FALSE COLOR COMPOSITE------
fcc_visParams = {'min': 0,
                 'max': 0.5,
                 'gamma': [0.95, 1.1, 1],
                 'bands': ['B5', 'B4', 'B3']}

# ----TRUE COLOR COMPOSITE------
tcc_visParams = {'min': 0.0,
                 'max': 0.4,
                 'bands':['B4', 'B3', 'B2']}

# Add the tile layer of interest to the folium map
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
# Specify default location for map
# Specify basemap: Folium defaults to openstreetmap basemap if a tile isn't specified
imageryMap = folium.Map(location=[42.2626,-71.8023], zoom_start=7, height=500, tiles='Stamen Terrain')
imageryMap.add_ee_layer(fcc, fcc_visParams, 'False Color Composite')
imageryMap.add_ee_layer(fcc, tcc_visParams, 'True Color Composite')

# Add a layer control panel to the map
# Allows you to turn layers on and off
imageryMap.add_child(folium.LayerControl())

# Display the map
display(imageryMap)